In [ ]:
# Google Collab Requirements / Checks
from google.colab.patches import cv2_imshow # Change this before running elsewhere!!

from google.colab import drive
drive.mount('/content/gdrive')

# Imports
import cv2
import os
import glob
import math
import numpy as np
import pandas as pd
import copy

from PIL import Image
from torchvision import transforms
from shapely.geometry import Polygon

Mounted at /content/gdrive


In [ ]:
# Load manual ms coco dataset and bounding boxes
# Varun's Directory
dir_uri = '/content/gdrive/MyDrive/University of Wisconsin-Madison/2021 Fall/cs762'
# Devesh's Directory
# dir_uri = '/content/gdrive/MyDrive'
proj_dir = dir_uri + '/CS762_Deep_Learning_Project'

# Images
dataset_folder = dir_uri + '/CS762_Deep_Learning_Project/datasets/Manual_MSCOCO' 
image_names = []
for filename in os.listdir(dataset_folder):
  image_names.append(filename)

image_names.sort(key=lambda x: int(x.split('-')[0]))

# Load bbox's
bbox_path = proj_dir + '/fullBBOX.xlsx'
bbox_df = pd.read_excel(bbox_path)
all_boxes = bbox_df.to_numpy()

detr_bbox_path = proj_dir + '/detr_bbox.csv'
detr_bbox_df = pd.read_csv(detr_bbox_path)
detr_boxes = detr_bbox_df.to_numpy()

yolo_bbox_path = proj_dir + '/yolo_bbox.csv'
yolo_bbox_df = pd.read_csv(yolo_bbox_path)
yolo_boxes = yolo_bbox_df.to_numpy()

deit_bbox_path = proj_dir + '/deit_bbox.csv'
deit_bbox_df = pd.read_csv(deit_bbox_path)
deit_boxes = deit_bbox_df.to_numpy()

resnet_bbox_path = proj_dir + '/resnet_bbox.csv'
resnet_bbox_df = pd.read_csv(resnet_bbox_path)
resnet_boxes = resnet_bbox_df.to_numpy()

In [ ]:
def resizeBbox(orig_bbox, orig_img_size, new_img_size):
    orig_h = orig_img_size[0]
    orig_w = orig_img_size[1]
    new_h = new_img_size[0]
    new_w = new_img_size[1]

    h_ratio = new_h / orig_h
    w_ratio = new_w / orig_w

    # new_bbox = copy.deepcopy(orig_bbox)
    # new_bbox[2] = orig_bbox[2] * w_ratio
    # new_bbox[3] = orig_bbox[3] * h_ratio
    # new_bbox[4] = orig_bbox[4] * w_ratio
    # new_bbox[5] = orig_bbox[5] * h_ratio
    # new_bbox[6] = orig_bbox[6] * w_ratio
    # new_bbox[7] = orig_bbox[7] * h_ratio
    # new_bbox[8] = orig_bbox[8] * w_ratio
    # new_bbox[9] = orig_bbox[9] * h_ratio

    new_bbox = [
        orig_bbox[0] * w_ratio,
        orig_bbox[1] * h_ratio,

        orig_bbox[2] * w_ratio,
        orig_bbox[3] * h_ratio,

        orig_bbox[4] * w_ratio,
        orig_bbox[5] * h_ratio,

        orig_bbox[6] * w_ratio,
        orig_bbox[7] * h_ratio
    ]

    return new_bbox

def get_box( name, boxes ):
  for box in boxes:
    if box[0] == name:
      return box

  return None

In [ ]:
detr_transform = transforms.Compose([
    transforms.Resize(800),
])

yolo_image_shape = ( 416, 416, 3 )
# resnet_image_shape = ( 224, 224, 3 )
# deit_image_shape = ( 224, 224, 3 )
classifier_image_shape = ( 224, 224, 3 )

In [ ]:
headers = ['image_name', 'IOU', 'delta_top', 'delta_bottom', 'delta_right', 'delta_left', 'center_dist']
detr_output = proj_dir + '/Q2/q2_detr_iou.csv'
yolo_output = proj_dir + '/Q2/q2_yolo_iou.csv'
deit_output = proj_dir + '/Q2/q2_deit_iou.csv'
resnet_output = proj_dir + '/Q2/q2_resnet_iou.csv'

boxes_to_analyze = {
    'detr': ( detr_boxes, detr_output ),
    'yolo': ( yolo_boxes, yolo_output ),
    'deit': ( deit_boxes, deit_output ),
    'resnet': ( resnet_boxes, resnet_output )
}

for box_type, box_data in boxes_to_analyze.items():
  boxes = box_data[0]
  output_file = box_data[1]

  model = box_type
  data = []

  images_to_check = [54]
  for i in range( len( boxes ) ):
    # Get boxes
    box_data = boxes[i]
    image_num = box_data[0]

    # Print
    print( 'Running for model {} on box {}, image num {}, original image name {}'.format( model, i, image_num, image_names[image_num] ) )

    image = Image.open( dataset_folder + '/' + image_names[image_num] ).convert('RGB')
    orig_img_shape = np.array( image ).shape

    new_image = None
    new_img_shape = None
    if model == 'detr':
      new_image = detr_transform( image )
      new_img_shape = np.array( new_image ).shape
    elif model == 'yolo':
      new_img_shape = yolo_image_shape
    else:
      new_img_shape = classifier_image_shape
    
    orig_img_shape = ( orig_img_shape[0], orig_img_shape[1] )
    new_img_shape = ( new_img_shape[0], new_img_shape[1] )

    orig_box = get_box( image_num, all_boxes )[2:10]
    # orig_box = resizeBbox( orig_box, orig_img_shape, new_img_shape )
    
    gradcam_box = box_data[2:10]
    gradcam_box = resizeBbox( gradcam_box, new_img_shape, orig_img_shape )

    # print(orig_box)
    # print(gradcam_box)

    # Compute IOU
    orig_poly = Polygon([(orig_box[0], orig_box[1]), (orig_box[2], orig_box[3]), (orig_box[4], orig_box[5]), (orig_box[6], orig_box[7])])
    new_poly = Polygon([(gradcam_box[0], gradcam_box[1]), (gradcam_box[2], gradcam_box[3]), (gradcam_box[4], gradcam_box[5]), (gradcam_box[6], gradcam_box[7])])
    
    IOU = orig_poly.intersection(new_poly).area / orig_poly.union(new_poly).area

    # Deltas
    delta_top = abs( orig_box[1] - gradcam_box[1] )
    delta_bottom = abs( orig_box[7] - gradcam_box[7] )
    delta_right = abs( orig_box[2] - gradcam_box[2] )
    delta_left = abs( orig_box[0] - gradcam_box[0] )

    orig_center_x = ( orig_box[1] + orig_box[3] + orig_box[5] + orig_box[7] ) / 4
    orig_center_y = ( orig_box[0] + orig_box[2] + orig_box[4] + orig_box[6] ) / 4

    new_center_x = ( gradcam_box[1] + gradcam_box[3] + gradcam_box[5] + gradcam_box[7] ) / 4
    new_center_y = ( gradcam_box[0] + gradcam_box[2] + gradcam_box[4] + gradcam_box[6] ) / 4

    center_dist = math.sqrt( ( orig_center_x - new_center_x ) ** 2 + ( orig_center_y - new_center_y ) ** 2 )

    # Store data
    data.append( [ image_names[image_num], IOU, delta_top, delta_bottom, delta_right, delta_left, center_dist ] )

  # Write current data to output file
  df = pd.DataFrame ( data, columns=headers )
  df.to_csv( output_file )

Running for model detr on box 0, image num 0, original image name 0-bicycle.jpeg
Running for model detr on box 1, image num 1, original image name 1-bicycle.jpeg
Running for model detr on box 2, image num 2, original image name 2-bicycle.jpeg
Running for model detr on box 3, image num 3, original image name 3-bicycle.jpeg
Running for model detr on box 4, image num 4, original image name 4-bicycle.jpeg
Running for model detr on box 5, image num 5, original image name 5-person.png
Running for model detr on box 6, image num 6, original image name 6-person.jpeg
Running for model detr on box 7, image num 7, original image name 7-person.jpeg
Running for model detr on box 8, image num 8, original image name 8-person.jpeg
Running for model detr on box 9, image num 9, original image name 9-person.jpeg
Running for model detr on box 10, image num 10, original image name 10-car.jpeg
Running for model detr on box 11, image num 11, original image name 11-car.jpeg
Running for model detr on box 12, im